# Prompting Best Practices with Amazon Nova Models

The effectiveness of prompts depends greatly on the quality of information provided and the craftsmanship of the prompt itself. Effective prompts may include instructions, questions, contextual details, inputs, and examples to guide the model and enhance result quality. 

This notebook explores strategies and techniques for optimizing the performance of Amazon Nova models. The methods presented can be combined in various ways to maximize their effectiveness. We encourage experimentation to identify approaches best suited to your specific needs.

For more comprehensive guidance, refer to the [Amazon Nova prompting documentation](https://docs.aws.amazon.com/nova/latest/userguide/prompting-text-understanding.html).

## Before Starting Prompt Engineering

Before diving into prompt engineering, it's recommended to establish:

1. **Define Your Use Case** along four key dimensions:
   - **Task**: What specific outcome do you want to achieve?
   - **Role**: What persona should the model adopt?
   - **Response Style**: What format or structure should the output follow?
   - **Instructions**: What guidelines should the model adhere to?

2. **Success Criteria**: Clearly define what constitutes a successful response. This might include:
   - Qualitative measures (format, factuality, faithfulness)
   - Quantitative metrics (length requirements, BLEU/Rouge scores)
   - Specific output requirements

3. **Draft Prompt**: Create an initial prompt to begin the iterative optimization process.

In [ ]:
# read in variables and config from previous notebook
%store -r

In [ ]:
from IPython.display import display, Markdown
import base64
import boto3
import json

# Set up the boto3 client with the proper region
boto3.setup_default_session(region_name=region_name)
client = boto3.client("bedrock-runtime")


def call_nova(
    model,
    messages,
    system_message="",
    streaming=False,
    max_tokens=512,
    temp=0,
    top_p=0.1,
    top_k=1,
    tools=None,
    stop_sequences=[],
    verbose=False,
):
    """Call Amazon Nova models with various parameters.
    
    Args:
        model (str): The model ID to use
        messages (list): List of message objects with role and content
        system_message (str, optional): System prompt. Defaults to "".
        streaming (bool, optional): Whether to use streaming API. Defaults to False.
        max_tokens (int, optional): Maximum tokens to generate. Defaults to 512.
        temp (float, optional): Temperature parameter. Defaults to 0.
        top_p (float, optional): Top-p parameter. Defaults to 0.1.
        top_k (int, optional): Top-k parameter. Defaults to 1.
        tools (list, optional): List of tool specifications. Defaults to None.
        stop_sequences (list, optional): List of stop sequences. Defaults to [].
        verbose (bool, optional): Whether to print request body. Defaults to False.
        
    Returns:
        tuple or stream: Model response and content text if not streaming, else stream
    """
    # Prepare system prompt
    system_list = [{"text": system_message}]
    
    # Prepare inference parameters
    inf_params = {
        "max_new_tokens": max_tokens,
        "top_p": top_p,
        "top_k": top_k,
        "temperature": temp,
        "stopSequences": stop_sequences,
    }
    
    # Build request body
    request_body = {
        "messages": messages,
        "system": system_list,
        "inferenceConfig": inf_params,
    }
    
    # Add tool configuration if provided
    if tools is not None:
        tool_config = []
        for tool in tools:
            tool_config.append({"toolSpec": tool})
        request_body["toolConfig"] = {"tools": tool_config}
    
    if verbose:
        print("Request Body", request_body)
    
    if not streaming:
        # Use synchronous API
        response = client.invoke_model(modelId=model, body=json.dumps(request_body))
        model_response = json.loads(response["body"].read())
        return model_response, model_response["output"]["message"]["content"][0]["text"]
    else:
        # Use streaming API
        response = client.invoke_model_with_response_stream(
            modelId=model, body=json.dumps(request_body)
        )
        return response["body"]


def get_base64_encoded_value(media_path):
    """Convert media file to base64 encoded string.
    
    Args:
        media_path (str): Path to the media file
        
    Returns:
        str: Base64 encoded string
    """
    with open(media_path, "rb") as media_file:
        binary_data = media_file.read()
        base_64_encoded_data = base64.b64encode(binary_data)
        base64_string = base_64_encoded_data.decode("utf-8")
        return base64_string


def print_output(content_text):
    """Display model output as Markdown.
    
    Args:
        content_text (str): Text to display
    """
    display(Markdown(content_text))


def validate_json(json_string):
    """Validates if a string is properly formatted JSON.
    
    Args:
        json_string (str): String to validate as JSON
        
    Returns:
        dict or None: Parsed JSON object if valid, None if invalid
    """
    try:
        # Attempt to parse the JSON string
        parsed_json = json.loads(json_string)
        
        # If successful, return the parsed JSON
        print("Valid JSON")
        return parsed_json
        
    except json.JSONDecodeError as e:
        # If parsing fails, print an error message
        print(f"Invalid JSON: {e}")
        
        # Print the location of the error
        print(f"Error at line {e.lineno}, column {e.colno}")
        
        # Return None to indicate failure
        return None

## Structured Outputs

For many applications, it's crucial to ensure that model responses follow a specific output format that works seamlessly with downstream processing. This is particularly important for automated workflows where inputs and outputs must adhere to strict formatting requirements.

### Techniques for Generating Structured Output

1. **Output Schema Specification**: Explicitly define the expected output structure in your prompt
2. **Elimination of Preambles**: Instruct the model to output only the required format without explanatory text
3. **Response Prefilling**: Guide the model's initial response format

### Using Response Prefilling

A particularly effective technique is to "nudge" the model's response by prefilling the assistant's content. This approach allows you to:

1. **Direct the Model's Behavior**: Put words in the model's mouth to guide its response
2. **Skip Unwanted Preamble**: Bypass explanatory text that might precede the desired output
3. **Enforce Output Format**: Ensure specific formats like JSON or XML

For example, prefilling with `{` or ``\`json` can guide the model to immediately begin generating JSON without explanatory text.

**Pro Tip**: When specifically extracting JSON, a common pattern is to prefill with ``\`json` and add a stop sequence on ``\`` to ensure the output is programmatically parseable.

In [ ]:
# Define an unoptimized prompt for getting camera information
unoptimized_prompt = """Provide details about the best selling full-frame cameras in past three years.
Answer in JSON format with keys like name, brand, price and a summary.
"""

# Create a messages object for the model
messages = [
    {"role": "user", "content": [{"text": unoptimized_prompt}]},
]

In [ ]:
model_response, content_text = call_nova(LITE_MODEL_ID, messages)
print("\n[Response Content Text]")
print("-" * 40)
print(content_text)
print("-" * 40)

### Improving Schema Definition with Data Types and Prefill

Let's enhance our approach by:
1. Adding explicit schema definitions with correct data types
2. Using response prefilling to ensure proper JSON formatting
3. Adding a stop sequence to control output length

In [ ]:
# Define an optimized prompt with explicit schema definition
optimized_prompt = """Provide 5 examples of the best selling full-frame cameras in past three years.
Follow the Output Schema as described below:
Output Schema:
{
"name" : <string, the name of product>,
"brand" : <string, the name of product>,
"price" : <integer price>,
"summary": <string, the product summary>
}
Only Respond in Valid JSON, without Markdown
"""

# Create a messages object with prefilled assistant response to guide the model
messages = [
    {"role": "user", "content": [{"text": optimized_prompt}]},
    {"role": "assistant", "content": [{"text": "```json"}]},
]

# Call the model with a stop sequence to ensure proper JSON formatting
model_response, content_text = call_nova(LITE_MODEL_ID, messages, stop_sequences=["]"])
print("\n[Response Content Text]")
print("-" * 40)
print(content_text)
print("-" * 40)

# Validate that the response is proper JSON
print("Testing valid JSON:")
parsed_json = validate_json(content_text)
if parsed_json:
    print(parsed_json)

## Few-Shot Examples

Including examples in your prompt can significantly improve the model's ability to generate responses aligned with your expectations. This technique, known as "few-shot prompting," helps the model understand the desired output format, style, and reasoning pattern.

### Benefits of Few-Shot Examples

- **Consistent Responses**: Creates uniformity in style and format
- **Enhanced Performance**: Reduces misinterpretation of instructions
- **Reduced Hallucinations**: Provides concrete guidance for outputs

### Characteristics of Effective Examples

1. **Diversity**: Include a range of examples that represent various use cases (from common to edge cases)
2. **Complexity Alignment**: Match the complexity of examples to the target task
3. **Relevance**: Ensure examples directly relate to the problem at hand

**Advanced Tip**: For dynamic applications, consider implementing a RAG-based system that selects relevant examples based on similarity to the current query.

In [ ]:
# Define a zero-shot prompt for sentiment classification
no_shot = """Your task is to Classify the following texts into the appropriate sentiment classes. The categories to classify are:

Sentiment Classes:
- Positive
- Negative
- Neutral

Query:
Input: The movie makes users think about their lives with the teenagers while still making audience unclear on the storyline.
"""

# Create a messages object
messages = [{"role": "user", "content": [{"text": no_shot}]}]

# Call the model with the zero-shot prompt
model_response, content_text = call_nova(LITE_MODEL_ID, messages)
print("\n[Response Content Text]")
print("-" * 40)
print_output(content_text)
print("-" * 40)

### Zero-Shot Sentiment Classification

Let's start with a simple sentiment classification task without providing examples. Notice how the model responds without clear guidance on output format:

In [ ]:
# Define a few-shot prompt with 4 examples for sentiment classification
four_shot = """Your task is to Classify the following texts into the appropriate sentiment classes. The categories to classify are:

Sentiment Classes:
- Positive
- Negative
- Neutral

Please refer to some examples mentioned below.

## Examples
### Example 1
Input: The movie was crazy good! I loved it
Output: Positive
Explaination: The text said "good" and "loved" so its positive

### Example 2
Input: The movie was scary and I got scared!
Output: Neutral
Explaination: The text said "scary" and "scared" which can be both positive and negative depending on people who like scary movies or one who hate

### Example 3
Input: The movie was pathetic not worth the time or money!
Output: Negative
Explaination: The text said "pathetic" and "not worth" which is negative sentiment

### Example 4
Input: The movie had some plots which were interesting and great while there were some gaps which needed more drama!
Output: Neutral
Explaination: The text said "interesting and great" and "some gaps" making it a mixed opinion hence neutral

Query:
Input: The movie makes users think about their lives with the teenagers while still making audience unclear on the storyline.
"""

# Create a messages object
messages = [{"role": "user", "content": [{"text": four_shot}]}]

# Call the model with the few-shot prompt
model_response, content_text = call_nova(LITE_MODEL_ID, messages)
print("\n[Response Content Text]")
print("-" * 40)
print_output(content_text)
print("-" * 40)

In [ ]:
# Define a prompt without Chain of Thought (CoT) reasoning
no_cot = """You are a project manager for a small software development team tasked with launching a new app feature.
You want to streamline the development process and ensure timely delivery. Draft a project plan
"""

# Create a messages object
messages = [{"role": "user", "content": [{"text": no_cot}]}]

# Call the model with the no-CoT prompt
model_response, content_text = call_nova(LITE_MODEL_ID, messages, max_tokens=1024)
print("\n[Response Content Text]")
print("-" * 40)
print_output(content_text)
print("-" * 40)

## Chain of Thought (CoT) Reasoning

Chain of Thought prompting is a technique that encourages the model to work through a problem step-by-step, showing its reasoning process before arriving at a conclusion. This approach often leads to more accurate results, especially for complex tasks requiring multi-step reasoning.

Let's first examine a response without explicit CoT guidance:

In [ ]:
# Define a prompt with guided Chain of Thought (CoT) reasoning
guided_cot = """You are a project manager for a small software development team tasked with launching a new app feature.
You want to streamline the development process and ensure timely delivery.
Your task is to draft a project plan.

But first do some thinking on how you want to structure and go through below questions before starting the draft.
Please follow these steps:
1. Think about who the audience is (this is for CEOs, CTOs and other executives)
2. Think about what to start with
3. Think about what Challenges you want to solve with this app
4. Think about the Tasks that will be needed to be completed
5. Create Milestones
6. Monitor Progress and Optimize
Explain all your thinking in <thinking></thinking> XML Tags and then write the final copy of project plan for executives in <project_plan></project_plan> XML Tag.

Output Schema:
<thinking>
( thoughts to above questions)
</thinking>
<project_plan>
( project plan)
</project_plan>
"""

# Create a messages object
messages = [{"role": "user", "content": [{"text": guided_cot}]}]

# Call the model with the guided CoT prompt
model_response, content_text = call_nova(LITE_MODEL_ID, messages, max_tokens=2048)
print("\n[Response Content Text]")
print("-" * 40)
print_output(content_text)
print("-" * 40)

# Conclusion

This notebook has demonstrated several key prompting best practices that significantly enhance the performance and output quality of Amazon Nova models. By implementing these techniques, you can achieve more precise, consistent, and useful responses tailored to your specific requirements.

## Summary of Techniques Explored

### 1. Structured Outputs
- **Output Schema Definition:** Providing explicit JSON schema definitions with specific data types guides the model to produce properly formatted outputs.
- **Prefill Technique:** Using assistant prefill (`"```json"`) combined with appropriate stop sequences helps bypass preamble text and ensures clean, parseable JSON.
- **Benefits:** Consistent data structure for downstream processing, reduced parsing errors, and improved automation capabilities.

### 2. Few-Shot Examples
- **Example-Based Learning:** Providing diverse, relevant examples helps the model understand expected patterns and response styles.
- **Format Consistency:** Examples establish a clear template for responses, leading to more uniform outputs.
- **Demonstrating Edge Cases:** Good examples showcase both common scenarios and edge cases, improving model robustness.
- **Benefits:** More predictable outputs, reduced ambiguity, and higher accuracy across various inputs.

### 3. Chain of Thought (CoT) Reasoning
- **Guided Thinking Process:** Structured thinking steps help the model break down complex tasks into manageable components.
- **XML Tags for Separation:** Using tags like `<thinking>` and `<project_plan>` separates internal reasoning from final outputs.
- **Step-by-Step Instructions:** Clear, sequential instructions guide the model through logical reasoning paths.
- **Benefits:** More thoughtful responses, better handling of complex scenarios, and clearer separation between reasoning and final output.

## Nest steps

For more Nova specific prompting guide, please check out [Nova user guide](https://docs.aws.amazon.com/nova/latest/userguide/prompting.html)